In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize


In [5]:

# Load data
df2 = pd.read_excel('Datasets/Nepal Master Sheet.xlsx', sheet_name='Final_compiled')
df = df2.copy()



# Define Traverse assignment function (same as before)
def assign_traverse(gns):
    if not isinstance(gns, str):  # Handle non-string values
        return None
    gns = gns.split('22')[0].split('23')[0].strip("'").strip('"')
    if gns.startswith("S1"):
        return "Traverse 1*" if gns in ["S1m", "S1i"] else "Traverse 1"
    elif gns.startswith("S2"):
        return "Traverse 2"
    elif gns.startswith("S3"):
        if gns in ["S3k", "S3m", "S3u", "S3s", "S3ag", "S3ad"]:
            return "Traverse 4"
        elif gns in ["S3y", "S3ae"]:
            return "Traverse 3*"
        return "Traverse 3"
    elif gns.startswith("S4"):
        return "Traverse 5*" if gns in ["S4m", "S4l"] else "Traverse 5"
    return None



df["Traverse"] = df["GNS"].apply(assign_traverse)
df = df[df["Traverse"] == "Traverse 3"]

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/Nepal Master Sheet.xlsx'

In [ ]:



# Convert Li concentrations to millimolar
df['Li_mM'] = df['Li_ppm'] / 6.94

# Convert Li concentrations to nanomolar
df['Li_nM'] = df['Li_ppm'] * 1000000 / 6.94

# Convert Li concentrations to mol/m^3 (from nM)
df['Li_mol_m3'] = df['Li_nM'] * 1e-6  # Convert nM to mol/m^3


In [ ]:

# Normalize z': Assign z'=1 to the maximum concentration
min_Li = df["Li_mol_m3"].min()
max_Li = df["Li_mol_m3"].max()

df["z'"] = (df["Li_mol_m3"] - min_Li) / (max_Li - min_Li)






# Reaction rate constant k (log10 form) and unit conversion
log_k = -11.2  # Example log10 k value
k = np.exp(log_k)  # Convert to mol/m^2/s
A_s = 1  # Assume unit specific surface area for simplicity



In [ ]:


# Function to calculate N_D
def calculate_ND(phi, t, C0, k, A_s):
    """
    Calculate the Damköhler number N_D.
    phi: porosity
    t: time (h/w, in seconds)
    C0: initial concentration (in mol/m^3)
    k: reaction rate constant (in mol/m^2/s)
    A_s: specific surface area (in m^2/m^3)
    """
    return t * k * A_s / (phi * C0)



def EE(z, N_D, f, C0):
    """
    Explicit Euler method to solve for C' as a function of z'.
    
    Parameters:
        z: array of nondimensional distances (z')
        N_D: Damköhler number
        f: reaction fraction
        C0: initial concentration (C' at z'=0)
    
    Returns:
        C: array of concentrations at each z'
    """
    dz = z[1] - z[0]  # Step size in nondimensional distance
    C = np.zeros_like(z)  # Initialize concentration array
    C[0] = C0  # Set initial concentration
    
    for i in range(1, len(z)):
        # Update concentration using Explicit Euler formula
        C[i] = C[i-1] + dz * N_D * (1 - f)
    
    return C




In [ ]:

# Objective function for optimization
def objective(params):
    phi, t = params  # Unpack porosity and time (h/w)
    f = 0.5  # Known fraction of reaction
    z = np.linspace(0, 1, 100)  # Nondimensional grid
    C0 = df["Li_mol_m3"].min()  # Initial concentration in mol/m^3
    N_D = calculate_ND(phi, t, C0, k, A_s)  # Calculate N_D
    C_model = EE(z, N_D, f, C0)
    C_model_interp = np.interp(df["z'"], z, C_model)  # Interpolate to match data
    return np.sum((C_model_interp - df["Li_mol_m3"])**2)  # Minimize squared error



In [ ]:

# Initial guesses for porosity and time (h/w)
initial_guess = [0.07, 25 * 365 * 24 * 3600]  # 25 years in seconds



# Perform optimization
result = minimize(objective, initial_guess, bounds=[(0.01, 0.1), (1e7, 1e10)])
phi_fit, t_fit = result.x



# Simulate with fitted parameters
C0_fit = df['Li_mol_m3'].min()
N_D_fit = calculate_ND(phi_fit, t_fit, C0_fit, k, A_s)
z_fine = np.linspace(0, 1, 100)
C_fit = EE(z_fine, N_D_fit, 0.5, C0_fit)



In [ ]:

# Plot results
plt.figure(figsize=(8, 6))
plt.scatter(df["z'"], df["Li_mol_m3"] / 1e3, label="Data (mM)", color="blue")  # Convert data to mM for comparison
plt.plot(z_fine, C_fit, label=f"Fitted Model: $N_D={N_D_fit:.2e}$, $\\phi={phi_fit:.2f}, t={t_fit/365/24/3600:.1f}$ years", color="red")
plt.xlabel("Nondimensional Distance ($z'$)")
plt.ylabel("Concentration ($Li_{mol/m^3}$)")
plt.title("Steady-State Model Fitting")
plt.legend()
plt.grid()
plt.show()

# Print fitted parameters
print(f"Fitted Porosity (phi): {phi_fit:.2f}")
print(f"Fitted Time (t): {t_fit/365/24/3600:.1f} years")
print(f"Calculated N_D: {N_D_fit:.2e}")
